# ETL script

ETL Manual: https://teleperformance.sharepoint.com/:x:/r/sites/S.DAF.Operations_Data_Analytics/Shared%20Documents/BAU%20Tasks%20and%20Procedures/P.TTOK.CONT/PROCEDURE%201_ETL.xlsx?d=w89a05f7c5bcf43158ff39cfa7d141eaf&csf=1&web=1&e=cVHTOW

## Libraries

In [26]:
import os
import pandas as pd
import glob
import numpy as np
import time
import shutil
import sys
import calendar
import re
import win32com.client
from pathlib import Path
from zipfile import ZipFile
from datetime import datetime, timedelta
import win32com.client as win32
import subprocess
import requests
import warnings
import win32com.client as w3c
warnings.filterwarnings('ignore')

# !!! IMP NOTE ( DONT FORGET TO CHANGE THE TIMEZONE WHEN THE HOUR CHANGE)!!!

Change Time Zone			for Portugal, if Winter: "UTC+0", if Summers: "UTC +1" (+01:00 Paris now)	

--------------------------------------------------------------------------------------------------------------------------

## !!!CHANGE THE  USER NAME, AND DOWNLOAD PATH:

In [27]:
#THE CODE SHOULD SHOW YOUR NAME:
username = os.getlogin()
username
print(f"Username: {username}")
#IF YOU DONT USE YOUR DOWNLOAD FOLDER CHANGE THE BELOW STRING TO YOUR FOLDER:
folder="Downloads"
downloads_folder_path = fr"C:\\Users\\{username}\\{folder}"
print(f"Path:{downloads_folder_path}")

date_today= datetime.today().strftime('%d-%m-%Y')
log_df = pd.DataFrame(columns=['Filename', 'Date', 'Status', 'Message'])
log_df.drop(log_df.index, inplace=True)
print("---------------------------------------------")
print(log_df)

Username: sequeira.81
Path:C:\\Users\\sequeira.81\\Downloads
---------------------------------------------
Empty DataFrame
Columns: [Filename, Date, Status, Message]
Index: []


--------------------------------------------------------------------------------------------------------------------------

# HC tracker ( Friday: Current week / Monday: Previous week)

__Link:__ https://byteworks-va.bytelemon.com/hc/fte-tracker/bpo

__Dashboard:__ WORKFORCE

__TAB:__ FTE ENTRY - BPO Available FTE

__Filter:__ BPO Site: If friday extract the data from the current week, if Monday extract the data from last week,
On the left we have the month group, and then the week. 

__Filter( Advanced Filters // Category ):__  Select all the Categories and Apply

__Download:__ Click on on the 3 dots and then Export, a new page will open with an excel file and then we can export the file




In [3]:

Hc_Tracker_EMEA = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\40. FTE Tracker"

Hc_Tracker_onedrive= fr"C:\Users\\{username}\\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\40. FTE Tracker"

Hc_Tracker_files = [filename for filename in os.listdir(downloads_folder_path) if 
                  filename.startswith("available_fte_")]
print(Hc_Tracker_files)


for jj in Hc_Tracker_files:
    try:
        file_path = os.path.join(downloads_folder_path, jj)
        print(f"Reading file: {file_path}")
        u = pd.read_excel(file_path)
        # Get today's date
        today = datetime.today()
        # Get the weekday (0 is Monday, 6 is Sunday)
        weekday = today.weekday()
        print(weekday)
        # Calculate the ISO week
        year, week, _ = today.isocalendar()
        # If today is Friday, use the current week
        if weekday == 4:
            week_str = f"{year} W{week:02d}"
        # If today is Monday, use the previous week
        elif weekday == 0:
            # Subtract 7 days from today to get a date in the previous week
            last_week = today - timedelta(days=7)
            # Calculate the ISO week for the date in the previous week
            year, week, _ = last_week.isocalendar()
            week_str = f"{year} W{week:02d}"
      
        else:
            week_str = f"{year} W{week:02d}"

        print(f"Year week selected: {week_str}")
                    
        #Will verify how many columns has like "Target FTE", If more than 1 will keep the last columns and drop the rest.
        fte_cols = [col for col in u.columns if col.startswith('Target FTE')]
        if len(fte_cols) >1:
            fte_cols.sort()
            cols_drop = fte_cols[:-1]
            u=u.drop(columns=cols_drop)
        # Rename columna names like 'Unnamed' to blank value:    
        for col in u.columns:
            if col.startswith('Unnamed:'):
                u.rename(columns={col: ''}, inplace=True)
            # rename the column to Target FTE     
            elif col.startswith('Target FTE'):
                u.rename(columns={col:'Target FTE Final'},inplace= True)


        ff = f'site_management_{week_str}.xlsx'
        print(ff)
        
        u.to_excel(f"{Hc_Tracker_EMEA}/{ff}", index=False)
        u.to_excel(f"{Hc_Tracker_onedrive}/{ff}", index=False)

        
        
        # Update the log DataFrame in case of success
        log_df = log_df.append({'Filename': jj, 'Date': date_today, 'Status': 'Success', 'Message': 'File saved successfully'}, ignore_index=True)
    except Exception as e:
        # Update the log DataFrame in case of error
        log_df = log_df.append({'Filename': jj, 'Date': date_today, 'Status': 'Error', 'Message': str(e)}, ignore_index=True)


['available_fte_1725606337014916332.xlsx']
Reading file: C:\\Users\\sequeira.81\\Downloads\available_fte_1725606337014916332.xlsx
4
Year week selected: 2024 W36
site_management_2024 W36.xlsx


# TEAM FILE BK

____Note:__ Run the below code, this code will move the teamfile used on the last refresh and move it the backup folder, so we will able to have files to rollback in case:

In [4]:
#1 GROUP
#this code will move the team file from dim folder to a backup folder:
Date= datetime.now() - timedelta(1)
Date=Date.strftime('%Y-%m-%d')
TeamfileOnUse = r'\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\15. DIM files\TikTok Lisbon - Teamfile New - Teamfile Lisbon.csv'
FolderBk=fr'\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\15. DIM files\TeamFile BK\TikTok Lisbon - Teamfile New - Teamfile Lisbon-{Date}.csv'

try:
    # Move the files to the folder
    shutil.copy(TeamfileOnUse, FolderBk)
    

    print(f"Files moveed with success.")
except Exception as e:
    print("Fails", e)

Files moveed with success.


# TEAM FILE

__Step 1:__ Download the Team file from the below link:
https://teleperformance.sharepoint.com/:x:/r/sites/P.TTOK.CONTTEAMDATAGROUP/Shared%20Documents/General/Teamfile/teamfile_management.xlsx?d=wea7a73936c5d4b9590422a33d71c4b4d&csf=1&web=1&e=dfITMi



 __Step 2:__ Run the below code, code will verify the file that you download and validate if all columns that should be bool are bool, in case is not will print a text sayig it, and if all well will move a file to emea to the folder 15 DIM, with the name ( TikTok Lisbon - Teamfile New - Teamfile Lisbon (validate).csv).
 
 If the file have problems on these columns, if is empty the columns should not be a issue if is with #ref will crach the model. If have # ref please use the step 6 and inform the BO team, using the email script.
 
 __If all good go to the next step__

In [5]:
#2 GROUP
#The below code will take the file from sharepoint,verify is all the columns from the "Columns_name" varieable are bool
#if not the code will show the column with the wrong type and not move the file. 

df = pd.read_excel (fr'C:\\Users\\{username}\\{folder}\teamfile_management.xlsx', sheet_name = 'Teamfile')
df.columns=df.columns.str.title()

class NonBooleanColumnError(Exception):
    pass

def check_columns_for_bool(df, column_names):
    non_bool_columns = []

    for column_name in column_names:
        dtype = df[column_name].dtype
        if dtype != bool:
            non_bool_columns.append(column_name)

    if non_bool_columns:
        raise NonBooleanColumnError(f"Colunas não booleanas encontradas: {non_bool_columns},Tipo encontrado: {dtype}")

try:

    Columns_name=['Lark_Account_Provisioning','Lark_Account_Deprovisioning','Is_Active',
                    'Locker_Need_Termination','Last_Record']

    check_columns_for_bool(df, Columns_name)
    print("Todas as colunas são do tipo booleano.")
    df.to_csv (r'\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\15. DIM files\TikTok Lisbon - Teamfile New - Teamfile Lisbon (validate).csv',
           index =False)
    
except NonBooleanColumnError as e:
    print(f"Erro: {e}")
    

Todas as colunas são do tipo booleano.


__Just in case you need to change the excel file that validades the teamfile,the one in EMEA.__

__IMP: All changes on  the EMEA file, please change it on cloud too!!!!!__
https://teleperformance.sharepoint.com/sites/S.DAF.Operations_Data_Analytics/Shared%20Documents/Forms/AllItems.aspx?RootFolder=%2Fsites%2FS%2EDAF%2EOperations%5FData%5FAnalytics%2FShared%20Documents%2FP%2ETTOK%2ECONT%2F10%2E%20Data%20%26%20PBI%20management%2F1%2E%20Raw%2Ddata%20validation%20files

 __Step 3:__ Run the below code, this code will excute a refresh on the EMEA excel file that validates the Teamfile.  

In [6]:
#3 GROUP
gen_py_path = fr"C:\Users\\{username}\\AppData\Local\Temp\gen_py"
shutil.rmtree(gen_py_path, ignore_errors=True)
print('Starting the process....................')
xl = w3c.Dispatch("Excel.Application")
print('...1')
xl.Visible = False
print('...2')
xl.DisplayAlerts = False
print('...3')
print('Refreshing new data..............')
wb = xl.Workbooks.Open(r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\15. DIM files\Validate Team File - new.xlsx")
print(wb, 'File opened.....')
wb.RefreshAll()
xl.CalculateUntilAsyncQueriesDone()
print(wb,'File refreshed.....')
wb.Save()
print(wb,'File saved.....')
wb.Close(True)  
xl.Quit()
print('Refresh new data.... Done')


Starting the process....................
...1
...2
...3
Refreshing new data..............
<COMObject Open> File opened.....
<COMObject Open> File refreshed.....
<COMObject Open> File saved.....
Refresh new data.... Done


__Step 4:__ If the output on the previous code is "Refresh new data.... Done" you can  run the below code.

The code will read the "Validate Team File - new.xlsx", and show you as output the first sheet name. 

If you have the __NOK__ columns with all 0 jump to the __step 7__, if not go to the next step.

In [7]:
#4 GROUP
#THIS WILL READ THE VALIDATION FILE AFTER THE REFRESH, SO WE CAN SEE THE ERROS:
validation_teamfile =r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\15. DIM files\Validate Team File - new.xlsx"

pd.read_excel(validation_teamfile,skiprows=1,usecols=[1,2,3,4])


,Page,NOK,TO DO if Error
0,TEAM FILE,0,(1-TEAM FILE): The Logins returned are Logins ...
1,TEAM FILE,0,(2-End Date Validation):More than 1 end date b...
2,TEAM FILE,0,(3-Date Validation): Start date is smaller tha...
3,TEAM FILE,0,(4-Start Date Validation): Start date is empty
4,TEAM FILE,0,(5-Go-live Date Validation): Go-live date nee...
5,TEAM FILE,0,(6-Email duplicated Validation): Check if the ...


 __Step 5:__ If the above code gives you a __NOK__ you can use the below code to open the validate file an then check the Sheet name so you can indentify the error, so you can use the __step 6__ to email the BO team.


In [ ]:
# Specify the path to your excel file
file_path = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\15. DIM files\Validate Team File - new.xlsx"

# Open the file with the default application
if os.name == 'nt':  # For Windows
    os.startfile(file_path)


__Step 6:__ Once you identify what are the errors on the Teamfile, you can use the below code to send an email to BO team.

Change the variable "newmail.CC" in case you want to add your email, and you can change the variable newmail.Body to write a different mail.

__IMP: Do not remove the '''\ at the beginning and the last ''' as they ensure the formatting of the mail body remains intact.__ 

In [ ]:
#6 GROUP
import win32com.client
ol=win32com.client.Dispatch("outlook.application")
olmailitem=0x0 #size of the new email
newmail=ol.CreateItem(olmailitem)
newmail.Subject= 'Team File data issue'
newmail.To='TikTok-Backoffice-Lisbon@pt.teleperformance.com'
newmail.CC='Data-Analytics-TikTok-Lisbon@pt.teleperformance.com'
newmail.Body= '''\
Hello Team,

Could you please verify the following shortlogin "NDZA", has 2 end dates empty.


Thank you.

'''

# attach='C:\\Users\\admin\\Desktop\\Python\\Sample.xlsx'
# newmail.Attachments.Add(attach)

# To display the mail before sending it
#newmail.Display()

newmail.Send()

__IMP:__ Once the BO team fixe all the issues with the team file please do again the step ( 1, 2,3,4), if no noks, jump to the step 7.
    

__Step 7:__ If all good with the teamfile, run the 1 code group, the code will move the file with the right name for powerbi. 

Once this first group will show ("Files moveed with success."), run the 2 group that will move the file to sql folders.

In [10]:
#ETL 7 GROUP
#1 group:
df.to_csv (r'\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\15. DIM files\TikTok Lisbon - Teamfile New - Teamfile Lisbon.csv',
           index =False)
Teamfile_EMEA=r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\15. DIM files\TikTok Lisbon - Teamfile New - Teamfile Lisbon.csv"
Teamfile_datadrops= r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\08 - data engineering\Data Drops\P.TTOK.CONT\Dim_Agent"
Teamfile_Onedrive= fr"C:\Users\{os.getenv('USERNAME')}\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\15. DIM files"
print("Files moveed with success.")


Files moveed with success.


In [11]:
#ETL 8 GROUP
#2 group
try:
    # Move the files to the folder
    shutil.copy(Teamfile_EMEA, Teamfile_datadrops)
    shutil.copy(Teamfile_EMEA, Teamfile_Onedrive)

    print("Files moveed with success.")
except Exception as e:
    print("Fails", e)

Files moveed with success.


--------------------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------------------

## UTC Files

In [8]:
#1º GROUP
#This code will read the folder paths:

#EMEA Folders:
BPO_PEOPLE_EMEA = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\BPO_PEOPLE"
BPO_PEOPLE_MI_EMEA = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\BPO_PEOPLE_MI"
BPO_QUEUE_HOUR_EMEA = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\BPO_QUEUE HOUR"
BPO_QUEUE_DAILY_EMEA = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\BPO_QUEUE_DAILY"
BPO_QUEUE_VMS_EMEA = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\BPO_QUEUE_VMS"
INTEGRITY_EMEA = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\INTEGRITY"
MODERATION_STATS_EMEA = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\MODERATION_STATS"
MODERATION_STATS_HOUR_EMEA = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\MODERATION_STATS_HOUR"
BPO_QUEUE_HOUR_RR2_EMEA = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\BPO_QUEUE_HOUR_RR2.0"
BPO_QUEUE_DAILY_RR2_EMEA = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\BPO_QUEUE_DAILY_RR2.0"

#Data Drops (New paths) ) DE Team  SQL folders
BPO_PEOPLE_DROP = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\08 - data engineering\Data Drops\P.TTOK.CONT\BPOPeopleWorkhour_new"
BPO_QUEUE_HOUR_DROP =r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\08 - data engineering\Data Drops\P.TTOK.CONT\BPO_Queue_new"
BPO_QUEUE_VMS_DROP = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\08 - data engineering\Data Drops\P.TTOK.CONT\BPO_Queue_VMS"
INTEGRITY_DROP = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\08 - data engineering\Data Drops\P.TTOK.CONT\BPO_Queue_integrity"
MODERATION_STATS_HOUR_DROP =r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\08 - data engineering\Data Drops\P.TTOK.CONT\BPO_hour_global_moderation_stat"

#Databricks
BPO_PEOPLE_DATABRICKS= r"\\10.235.133.90\ptdoufs01Departments$\ITDEV\DataManagement\TTOK\bpopeopleworkhour"

#Data Sharepoint:
BPO_Onedrive= fr"C:\Users\\{username}\\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\BPO_PEOPLE"
INTEGRITY_Ondrive= fr"C:\Users\\{username}\\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\INTEGRITY"

#Validacao:
BPO_PEOPLE_Validacao: r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\Validation\BPO_People"
BPO_PEOPLE_Validacao_ADSO:r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\Validation\BPO_People_ADSO"
BPO_PEOPLE_Validacao_Integrity:r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\Validation\Integrity"
BPO_PEOPLE_Validacao_Integrity_MI:r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\Validation\Integrity_MI"
    
#Folders for validation code( The lines have a short_name)
folders2={
    'BPO_PEOPLE(3 em 3)download last 2 weeks data)': r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\Validation\BPO_People",
    'BPO_PEOPLE_ADSO(3 em 3)download last 2 weeks data)':r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\Validation\BPO_People_ADSO",
    'BPO_QUEUE_HOUR_RR1.0 (4 em 4)': r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\BPO_QUEUE HOUR",
    'BPO_QUEUE_HOUR_RR2.0(4 em 4)': r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\BPO_QUEUE_HOUR_RR2.0",
    'BPO_QUEUE_VMS(8 em 8)':r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\BPO_QUEUE_VMS",
    'MODERATION_STATS_HOUR_EMEA(4 em 4)':r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\MODERATION_STATS_HOUR",
    'BPO_QUEUE_DAILY_RR1.0(8 em 8)': r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\BPO_QUEUE_DAILY",
    'BPO_QUEUE_DAILY_RR2.0(8 em 8)': r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\BPO_QUEUE_DAILY_RR2.0",
    'MODERATION_STATS_EMEA(15 em 15)':r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\MODERATION_STATS",
    'INTEGRITY_EMEA(15 em 15)':r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\Validation\Integrity",
    'INTEGRITY_MI(15 em 15)':r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\Validation\Integrity_MI"}


#Folders for validation code( The lines have a short_name)
folders={
    'BPO_PEOPLE(3 em 3)download last 2 weeks data)': r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\BPO_PEOPLE",
    'BPO_PEOPLE_MI(3 em 3)download last 2 weeks data)':r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\BPO_PEOPLE_MI",
    'BPO_QUEUE_HOUR_RR1.0 (4 em 4)': r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\BPO_QUEUE HOUR",
    'BPO_QUEUE_HOUR_RR2.0(4 em 4)': r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\BPO_QUEUE_HOUR_RR2.0",
    'BPO_QUEUE_VMS(8 em 8)':r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\BPO_QUEUE_VMS",
    'MODERATION_STATS_HOUR_EMEA(4 em 4)':r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\MODERATION_STATS_HOUR",
    'BPO_QUEUE_DAILY_RR1.0(8 em 8)': r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\BPO_QUEUE_DAILY",
    'BPO_QUEUE_DAILY_RR2.0(8 em 8)': r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\BPO_QUEUE_DAILY_RR2.0",
    'MODERATION_STATS_EMEA(15 em 15)':r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\MODERATION_STATS",
    'INTEGRITY_EMEA(15 em 15)':r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\INTEGRITY"}

__This code will show the last file on the above folder, base on the file name.__

In [9]:
#2º GROUP
#THIS CODE WILL VERIFY FROM ABOVE FOLDERM WHAT IS THE LAST FILES ON EACH FOLDER:
def format_text(text, color_code, bold=False):
    if bold:
        return f"\033[1;{color_code}m{text}\033[0m"
    else:
        return f"\033[{color_code}m{text}\033[0m"
for short_name, folder_path in folders.items():
    # Checking if the path is a folder
    if os.path.isdir(folder_path):
        # List all the files
        files = os.listdir(folder_path)
        
        if files:
            # Find the most resent date base on the file name
            max_date = None
            max_date_file = None
            date_pattern = re.compile(r'_(\d{8})_(\d{8})')  # Expression of the date

            for file in files:
                date_matches = date_pattern.search(file)
                if date_matches:
                    start_date_str, end_date_str = date_matches.groups()
                    start_date = datetime.strptime(start_date_str, '%d%m%Y')
                    end_date = datetime.strptime(end_date_str, '%d%m%Y')

                    # Use the final date (end_date) as a criterion for the most recent date
                    if max_date is None or end_date > max_date:
                        max_date = end_date
                        max_date_file = file
                else:
                    # Handles the format "01082023_04082023" without the prefix before the dates ( BPO PEOPLE case)
                    date_matches = re.search(r'(\d{8})_(\d{8})', file)
                    if date_matches:
                        start_date_str, end_date_str = date_matches.groups()
                        start_date = datetime.strptime(start_date_str, '%d%m%Y')
                        end_date = datetime.strptime(end_date_str, '%d%m%Y')

                        if max_date is None or end_date > max_date:
                            max_date = end_date
                            
                            max_date_file = file

            if max_date_file:
                max_date_format = format_text(max_date_file, 31, bold=True)
                short_name_format = format_text(short_name, 31, bold=True)
                print(f"'{short_name_format}', the most recent file is: '{max_date_format}'")

'BPO_PEOPLE(3 em 3)download last 2 weeks data)', the most recent file is: 'WorkHour Indicators 03092024_05092024.csv'
'BPO_PEOPLE_MI(3 em 3)download last 2 weeks data)', the most recent file is: 'WorkHour Indicators 03092024_05092024.csv'
'BPO_QUEUE_HOUR_RR1.0 (4 em 4)', the most recent file is: 'LatSLA_Hourly_03092024_06092024.csv'
'BPO_QUEUE_HOUR_RR2.0(4 em 4)', the most recent file is: 'LatSLA_Hourly_03092024_06092024.csv'
'BPO_QUEUE_VMS(8 em 8)', the most recent file is: 'LatSLA_VMS_30082024_06092024.csv'
'MODERATION_STATS_HOUR_EMEA(4 em 4)', the most recent file is: 'hour_global_moderation_stat_03092024_06092024.csv'
'BPO_QUEUE_DAILY_RR1.0(8 em 8)', the most recent file is: 'LatSLA_30082024_06092024.csv'
'BPO_QUEUE_DAILY_RR2.0(8 em 8)', the most recent file is: 'LatSLA_30082024_06092024.csv'
'MODERATION_STATS_EMEA(15 em 15)', the most recent file is: 'global_moderation_stat_01092024_15092024.csv'
'INTEGRITY_EMEA(15 em 15)', the most recent file is: 'Integrity_01092024_15092024.csv

__DATAPOWER LINK:___ https://datapower-va.bytelemon.com/operating/bpo_site_dashboard
 -----------------------------------------------------------------------------------------------------     
 __On the Bot BPO_PEOPLE:__
 
 __IMP: We need to use the GCP accountand select all the departems and not the "MI" department__
  
 1 file need to contain 3 days of data // Hourly data// Exclude the MI department from the department filter on the top // Everyday download all current week and previous  week/

__Last Table:__ (  Moderation Raking  )										
		
        a. Filter( Column present: Workhours Indicators )
		c. Export Data:  Export Custom Subnit Data ( here you select the date range) 				
		D. File type : CSV				
 -----------------------------------------------------------------------------------------------------     				
 
  __On the Bot BPO_PEOPLE_MI:__
 
 __IMP: We need to use the MI account and select only MI departmente:__
 
 1 file need to contain 3 days of data // Hourly data// Everyday download all current week and previous  week/

__Last Table:__ (  Moderation Raking  )										
		
        a. Filter( Column present: Workhours Indicators )
		c. Export Data:  Export Custom Subnit Data ( here you select the date range) 				
		D. File type : CSV				
 -----------------------------------------------------------------------------------------------------     						
 
 __BPO_QUEUE HOUR_RR1.0:__
 
 1 file 4 days//Hourly data

__First table:__  (Queue Statistics Table )						
						
		a. Filter( Column present: basics Indicators )				
		b.Queue type RR1.0				
		c. Export Data:  Export Custom Subnit Data ( here you select the date range) 				
		D. File type : CSV
 -----------------------------------------------------------------------------------------------------       
  __BPO_QUEUE HOUR_RR2.0:__
 
 1 file 4 days//Hourly data
 
 __First table:__ (Queue Statistics Table )						
						
		a. Filter( Column present: basics Indicators )				
		b.Queue type RR2.0				
		c. Export Data:  Export Custom Subnit Data ( here you select the date range) 				
		D. File type : CSV
        
-----------------------------------------------------------------------------------------------------                  
 __BPO_QUEUE_VMS:__
 
 1 file 8 days//Hour data  
 
 __First table:__  (Queue Statistics Table )							
							
		a. Filter( Column present: basics Indicators )					
		b.Queue type VMS					
		c. Export Data:  Export Custom Subnit Data ( here you select the date range) 					
		D. File type : CSV
        
 -----------------------------------------------------------------------------------------------------     
 __MODERATION_STATS_HOUR:__
 
 1  File 4 days // Hour data 

__Last Table:__ (  Moderation Raking  )							
							
		a. Filter( Column present:  Basic Indicators ) /// Select Queue Data 					
		b. Select Data + Column preset: Basic  + Check the queue  box					
		c. Export Data:  Export Custom Subnit Data ( here you select the date range) 					
		D. File type : CSV		

 -----------------------------------------------------------------------------------------------------     
 __BPO_QUEUE_DAILY_RR1.0:__
 
 1 file 8 days//Daily data
 
 __First table:__  (Queue Statistics Table )						

		a. Filter( Column present: basics Indicators )				
		b.Queue type RR1.0				
		c. Export Data: Export Page Unit Data  		
		D. File type : CSV	
        
 -----------------------------------------------------------------------------------------------------            
  __BPO_QUEUE_DAILY_RR2.0:__
 
 1 file 8 days//Daily data
 
 __First table:__  (Queue Statistics Table )						
						
		a. Filter( Column present: basics Indicators )				
		b.Queue type RR2.0				
		c. Export Data: Export Page Unit Data  		
		D. File type : CSV	
        
 
        
 -----------------------------------------------------------------------------------------------------     


  __MODERATION_STATS:__
  
  1 File (15 em 15) // Daily data      
 
 __Last Table:__ (  Moderation Raking  )						

		a. Filter( Column present:  Basic Indicators ) /// Select Queue Data 				
		b. Select Data + Column preset: Basic  + Check the queue  box				
		c. Export Data:  Export page unit Data				
		D. File type : CSV				
						
 -----------------------------------------------------------------------------------------------------    			
__INTEGRITY:__
 
 1 File (15 em 15)// Daily data
 
 __Last Table:__ (  Moderation Raking  )						
						
		a. Filter( Column present: Integrity ) /// Select Queue Data 				
		b. Select Data + Column preset: Basic  + Check the queue  box				
		c. Export Data:  Export page unit Data				
		D. File type : CSV							

 -----------------------------------------------------------------------------------------------------   
__INTEGRITY ( MI Account) :__

1 File (15 em 15)// Daily data

Last Table: ( Moderation Raking )

    a. Filter( Column present: Integrity ) /// Select Queue Data                 
    b. Select Data + Column preset: Basic  + Check the queue  box                
    c. Export Data:  Export page unit Data                
    D. File type : CSV       

In [12]:
#3º GROUP
#WILL SHOW THE NAME OF FILES THAT ARE ON OUR DOWNLOADS FOLDER BASE ON THE CONDITION OF THE FILE NAME"UTC"
utc_files = [filename for filename in os.listdir(downloads_folder_path) if 
                  filename.startswith("UTC")]
utc_files

['UTC+1：20240906080902.csv',
 'UTC+1：20240906080908.csv',
 'UTC+1：20240906081113.csv',
 'UTC+1：20240906081131.csv',
 'UTC+1：20240906081145.csv',
 'UTC+1：20240906081154.csv',
 'UTC+1：20240906081240.csv',
 'UTC+1：20240906081254.csv',
 'UTC+1：20240906081302.csv',
 'UTC+1：20240906081315.csv',
 'UTC+1：20240906081407.csv',
 'UTC+1：20240906081415.csv',
 'UTC+1：20240906081418.csv',
 'UTC+1：20240906081507.csv',
 'UTC+1：20240906081509.csv',
 'UTC+1：20240906081521.csv',
 'UTC+1：20240906081542.csv']

In [13]:
#4º GROUP
#Remove the files from folder BPO_QUEUE_HOUR_RR2_EMEA:
[os.remove(os.path.join(BPO_QUEUE_HOUR_RR2_EMEA, filename)) for filename in os.listdir(BPO_QUEUE_HOUR_RR2_EMEA)
 if filename.startswith("LatSLA_Hourly")]
#Remove the files from folder BPO_QUEUE_DAILY_RR2_EMEA:
[os.remove(os.path.join(BPO_QUEUE_DAILY_RR2_EMEA, filename)) for filename in os.listdir(BPO_QUEUE_DAILY_RR2_EMEA)
 if filename.startswith("LatSLA_")]
#Remove the files from folder BPO_PEOPLE_MI_EMEA:
[os.remove(os.path.join(BPO_PEOPLE_MI_EMEA, filename)) for filename in os.listdir(BPO_PEOPLE_MI_EMEA)
 if filename.startswith("WorkHour Indicators")]
#Remove the files from folder Valitaion BPO_PEOPLE_Validacao:
#[os.remove(os.path.join(BPO_PEOPLE_Validacao, filename)) for filename in os.listdir(BPO_PEOPLE_Validacao)
 #if filename.startswith("WorkHour Indicators")]
#Remove the files from folder Valitaion BPO_PEOPLE_Validacao_ADSO:
#[os.remove(os.path.join(BPO_PEOPLE_Validacao_ADSO, filename)) for filename in os.listdir(BPO_PEOPLE_Validacao_ADSO)
 #if filename.startswith("WorkHour Indicators")]
#Remove the files from folder Valitaion BPO_PEOPLE_Validacao_Integrity:
#[os.remove(os.path.join(BPO_PEOPLE_Validacao_Integrity, filename)) for filename in os.listdir(BPO_PEOPLE_Validacao_Integrity)
 #if filename.startswith("WorkHour Indicators")]
#Remove the files from folder Valitaion BPO_PEOPLE_Validacao_Integrity_MI:
#[os.remove(os.path.join(BPO_PEOPLE_Validacao_Integrity_MI, filename)) for filename in os.listdir(BPO_PEOPLE_Validacao_Integrity_MI)
 #if filename.startswith("WorkHour Indicators")]

[None, None, None, None]

In [14]:
#5º GROUP
#THIS CODE WILL TRANSFORM THE FILE IF NEED, CHANGE THE NAME,MOVE TO THE RIGHT FOLDER:

#Loop to read all the utc_files 
for i in utc_files:
 try:
    #drop the first row on all columns
    g = pd.read_csv(downloads_folder_path + '\\' + i, header=1,index_col = False)
    #rename all columns as Date, this is because bpo peopledata has other name.
    g = g.rename(columns = {g.columns[0]: 'Date'})
    #Variable with the columns name
    c = g.columns
#-----------------------------------Logic to get the start_date e end_date for filename----------------------------------------
# Get Start Date for Each File
    start_date = pd.to_datetime(g['Date'].min())
    
# Store the Year + Month of Start Date for End Of Month Calculation
# And store the end date for 4 days and 8 days type file
    start_date_year = start_date.year
    start_date_month = start_date.month
    end_date1 = start_date + timedelta(days = 3)
    end_date2 = start_date + timedelta(days = 7)
    end_date3 = start_date + timedelta(days = 2)

# Clean Start Date, End Date 4 Days, End Date 8 Days
# Clean End Date 1 Month on file specific step
    start_date = start_date.strftime('%d%m%Y')
    end_date1 = end_date1.strftime('%d%m%Y')
    end_date2 = end_date2.strftime('%d%m%Y')
    end_date3 = end_date3.strftime('%d%m%Y')

# Normalizes the Start and End Date formats
# Normalization of End Date Month on file specific Step
    if len(start_date) < 8:
        start_date = '0'+ start_date
    if len(end_date1) < 8:
        end_date1 = '0'+ end_date1
    if len(end_date2) < 8:
        end_date2 = '0'+ end_date2
    if len(end_date3) < 8:
        end_date3 = '0'+ end_date3
#-----------------------------------END of Logic to get the start_date e end_date for filename----------------------------------------        
#BPO_People HOUR (Need to do merge with BPO people MI)
 
    if 'Others Hours(h)' in list(c) and not g['Department Name'].str.contains('MI_TP_LIS').any():
        f = "WorkHour Indicators %s_%s.csv" %(start_date, end_date3)
        g = g.rename(columns = {g.columns[0]: 'Hour'})
        g.to_csv(BPO_PEOPLE_EMEA + '\\'+ f, index = False)
        print("1:WorkHour Indicators: " + i)
        log_df = log_df.append({'Filename': i, 'Date': start_date, 'Status': 'Success', 'Message': 'File copied successfully'}, ignore_index=True)
        
#BPO_People HOUR (MI ACCOUNT)

    elif 'Others Hours(h)' in list(c) and g['Department Name'].str.contains('MI_TP_LIS').any():
        f = "WorkHour Indicators %s_%s.csv" %(start_date, end_date3)
        g = g.rename(columns = {g.columns[0]: 'Hour'}) 
        g.to_csv(BPO_PEOPLE_MI_EMEA+ '\\'+ f, index = False)
        print("2:WorkHour Indicators_MI: " + i)
        log_df = log_df.append({'Filename': i, 'Date': start_date, 'Status': 'Success', 'Message': 'File copied successfully'}, ignore_index=True)        
        
           
 #BPO_QUEUE HOUR_RR1.0

    elif len(list(c)) == 23 and 'No. of Input' in list(c):
        f = 'LatSLA_Hourly_%s_%s.csv' %(start_date, end_date1)
        g.drop(['Ontime Output Completion SLA(%)'],axis=1, inplace=True)
        g.to_csv(BPO_QUEUE_HOUR_EMEA+ '\\' + f, index = False)
        print("3:LatSLA_Hourly: " + i)
        log_df = log_df.append({'Filename': i, 'Date': start_date, 'Status': 'Success', 'Message': 'File copied successfully'}, ignore_index=True)
        
 #BPO_QUEUE HOUR_RR2.0 

    elif len(list(c)) == 19 and 'No. of Output_RR2.0 High Priority' in list(c):
        f = 'LatSLA_Hourly_%s_%s.csv' %(start_date, end_date1)
        g.drop(['No.of High-pri. Incoming','No. of Output_RR2.0 High Priority','Ontime Output Completion SLA(%)'],axis=1, inplace= True)
        g.to_csv(BPO_QUEUE_HOUR_RR2_EMEA+ '\\' + f, index = False)
        print("4:LatSLA_Hourly: " + i)
        log_df = log_df.append({'Filename': i, 'Date': start_date, 'Status': 'Success', 'Message': 'File copied successfully'}, ignore_index=True)
       

 #BPO_QUEUE_DAILY_RR1.0
    
    elif len(list(c)) == 28 and 'No. of Input' in list(c):
        f = 'LatSLA_%s_%s.csv' %(start_date, end_date2)
        g.drop(['Ontime Output Completion SLA(%)'],axis=1, inplace=True)
        g.to_csv(BPO_QUEUE_DAILY_EMEA + '\\'+ f, index = False)
        print("5:LatSLA: " + i)
        log_df = log_df.append({'Filename': i, 'Date': start_date, 'Status': 'Success', 'Message': 'File copied successfully'}, ignore_index=True)
        
  #BPO_QUEUE_DAILY_RR2.0
    
    elif len(list(c)) == 24 and 'No. of Output_RR2.0 High Priority' in list(c):
        f = 'LatSLA_%s_%s.csv' %(start_date, end_date2)
        g.drop(['No.of High-pri. Incoming','No. of Output_RR2.0 High Priority','Ontime Output Completion SLA(%)'],axis=1,inplace=True)
        g.to_csv(BPO_QUEUE_DAILY_RR2_EMEA + '\\'+ f, index = False)
        print("6:LatSLA: " + i)
        log_df = log_df.append({'Filename': i, 'Date': start_date, 'Status': 'Success', 'Message': 'File copied successfully'}, ignore_index=True)
        
 #BPO_QUEUE_VMS

    elif 'No. of Achievable Output_VMS3' in list(c):
        f = 'LatSLA_VMS_%s_%s.csv' %(start_date, end_date2)
        g = g.rename(columns ={"Ontime Output":"Ontime Output_VMS3.0"})
        g.to_csv(BPO_QUEUE_VMS_EMEA + '\\'+ f, index = False)
        g.to_csv(BPO_QUEUE_VMS_DROP + '\\'+ f, index = False)
        print("7:LatSLA_VMS: " + i)
        log_df = log_df.append({'Filename': i, 'Date': start_date, 'Status': 'Success', 'Message': 'File copied successfully'}, ignore_index=True)
        
 #INTEGRITY

    elif len(list(c)) == 12 and 'No. of Deferment (Cases)' in list(c)and not g['Department name'].str.contains('MI_TP_LIS').any():
        final_day = calendar.monthrange(start_date_year, start_date_month)[1]
        len_start_date_month = str(start_date_month)
        if len(len_start_date_month) < 2:
            len_start_date_month = '0'+ len_start_date_month
        
        if (start_date[:2] < "16"):
            end_date3 = "%s%s%s" %("15", len_start_date_month, start_date_year)
        else:
            end_date3 = "%s%s%s" %(final_day, len_start_date_month, start_date_year)
        
        f = 'Integrity_%s_%s.csv' %(start_date, end_date3)
        g = g.rename(columns ={"No tag Rate(%)":"No-tag Rate(%)"})
        g.to_csv(INTEGRITY_EMEA + '\\'+ f, index = False)
        g.to_csv(INTEGRITY_DROP + '\\'+ f, index = False)
        
        print("8:Integrity: " + i)
        log_df = log_df.append({'Filename': i, 'Date': start_date, 'Status': 'Success', 'Message': 'File copied successfully'}, ignore_index=True)
  
 #INTEGRITY MI

    elif len(list(c)) == 12 and 'No. of Deferment (Cases)' in list(c) and g['Department name'].str.contains('MI_TP_LIS').any():
        final_day = calendar.monthrange(start_date_year, start_date_month)[1]
        len_start_date_month = str(start_date_month)
        if len(len_start_date_month) < 2:
            len_start_date_month = '0'+ len_start_date_month
        
        if (start_date[:2] < "16"):
            end_date3 = "%s%s%s" %("15", len_start_date_month, start_date_year)
        else:
            end_date3 = "%s%s%s" %(final_day, len_start_date_month, start_date_year)
        
        f = 'Integrity_MI_%s_%s.csv' %(start_date, end_date3)
        g = g.rename(columns ={"No tag Rate(%)":"No-tag Rate(%)"})
        g.to_csv(INTEGRITY_EMEA + '\\'+ f, index = False)
        g.to_csv(INTEGRITY_DROP + '\\'+ f, index = False)
        g.to_csv(INTEGRITY_Ondrive + '\\'+ f, index = False)
        #g.to_csv(INTEGRITY_Validacao + '\\'+ f, index = False)
        print("888:Integrity_MI: " + i)
        log_df = log_df.append({'Filename': i, 'Date': start_date, 'Status': 'Success', 'Message': 'File copied successfully'}, ignore_index=True)        
 #MODERATION_STATS

    elif len(list(c)) == 26 and 'Total Productive Hours(h)' in list(c):
        final_day = calendar.monthrange(start_date_year, start_date_month)[1]
        len_start_date_month = str(start_date_month)
        if len(len_start_date_month) < 2:
            len_start_date_month = '0'+ len_start_date_month
        
        if (start_date[:2] < "16"):
            end_date3 = "%s%s%s" %("15", len_start_date_month, start_date_year)
        else:
            end_date3 = "%s%s%s" %(final_day, len_start_date_month, start_date_year)
        
        f = 'global_moderation_stat_%s_%s.csv' %(start_date, end_date3)
        g.to_csv(MODERATION_STATS_EMEA + '\\'+ f, index = False)
        print("9:global_moderation_stat: " + i)
        log_df = log_df.append({'Filename': i, 'Date': start_date, 'Status': 'Success', 'Message': 'File copied successfully'}, ignore_index=True)

 #MODERATION_STATS_HOUR:

    elif len(list(c)) == 21 and 'Total Productive Hours(h)' in list(c):
        f = 'hour_global_moderation_stat_%s_%s.csv' %(start_date, end_date1)
        g.to_csv(MODERATION_STATS_HOUR_EMEA + '\\'+ f, index = False)
        g.to_csv(MODERATION_STATS_HOUR_DROP + '\\'+ f, index = False)
        print("10:hour_global_moderation_stat: " + i)
        log_df = log_df.append({'Filename': i, 'Date': start_date, 'Status': 'Success', 'Message': 'File copied successfully'}, ignore_index=True)
 
# Fail condition, to catch any file not complying with the rules above
    else:
        print("File Not Uploaded: " + i) 
 except Exception as e:
    log_df = log_df.append({'Filename': i, 'Date': date_today, 'Status': 'Error', 'Message': str(e)}, ignore_index=True)
        

2:WorkHour Indicators_MI: UTC+1：20240906080902.csv
1:WorkHour Indicators: UTC+1：20240906080908.csv
2:WorkHour Indicators_MI: UTC+1：20240906081113.csv
2:WorkHour Indicators_MI: UTC+1：20240906081131.csv
2:WorkHour Indicators_MI: UTC+1：20240906081145.csv
1:WorkHour Indicators: UTC+1：20240906081154.csv
3:LatSLA_Hourly: UTC+1：20240906081240.csv
4:LatSLA_Hourly: UTC+1：20240906081254.csv
1:WorkHour Indicators: UTC+1：20240906081302.csv
7:LatSLA_VMS: UTC+1：20240906081315.csv
5:LatSLA: UTC+1：20240906081407.csv
1:WorkHour Indicators: UTC+1：20240906081415.csv
6:LatSLA: UTC+1：20240906081418.csv
10:hour_global_moderation_stat: UTC+1：20240906081507.csv
9:global_moderation_stat: UTC+1：20240906081509.csv
8:Integrity: UTC+1：20240906081521.csv


----------------------------------------END of this code Group--------------------------------------------------------------


In [15]:
#6 Group
#THIS CODE WILL CONCAT THE FILES FROM THE BELOW FOLDER TOGETHER TO MERGE THE RR1.0 WITH RR2.0

pasta_A = BPO_QUEUE_DAILY_EMEA
pasta_B = BPO_QUEUE_DAILY_RR2_EMEA
#---------------------------------
pasta_C = BPO_QUEUE_HOUR_EMEA
pasta_D = BPO_QUEUE_HOUR_RR2_EMEA
#---------------------------------
pasta_E = BPO_PEOPLE_EMEA
pasta_F = BPO_PEOPLE_MI_EMEA
#----------------------------------

# Lista todos os arquivos em cada pasta
arquivos_A = os.listdir(pasta_A)
arquivos_B = os.listdir(pasta_B)
#________________________________
arquivos_C = os.listdir(pasta_C)
arquivos_D = os.listdir(pasta_D)
#________________________________
arquivos_E = os.listdir(pasta_E)
arquivos_F = os.listdir(pasta_F)

#-----------BPO QUEUE Daily( merge rr1.0 with rr2.0)----------------------
# Itera sobre os arquivos com o mesmo nome em ambas as pastas
for arquivo1 in set(arquivos_A) & set(arquivos_B):
    caminho_A = os.path.join(pasta_A, arquivo1)
    caminho_B = os.path.join(pasta_B, arquivo1)
# Lê os dataframes de ambos os arquivos
    df_A = pd.read_csv(caminho_A)
    df_B = pd.read_csv(caminho_B)
    df_concatenado = pd.concat([df_A, df_B], axis=0)
    
    caminho_saida = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\BPO_QUEUE_DAILY\{}".format(arquivo1)
    df_concatenado.to_csv(caminho_saida, index=False)
    
#-------------BPO QUEUE Hour( merge rr1.0 with rr2.0)---------------------------------------------------    
for arquivo2 in set(arquivos_C) & set(arquivos_D):
    caminho_C = os.path.join(pasta_C, arquivo2)
    caminho_D = os.path.join(pasta_D, arquivo2)
    # Lê os dataframes de ambos os arquivos
    df_C = pd.read_csv(caminho_C)
    df_D = pd.read_csv(caminho_D)
    df_concatenado1 = pd.concat([df_C, df_D], axis=0)
    
    caminho_saida = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\BPO_QUEUE HOUR\{}".format(arquivo2)
    df_concatenado1.to_csv(caminho_saida, index=False)
    
#----------------BPO PEOPLE ( merge Gcp data with MI data) --------------------------------------------------------------
for arquivo3 in set(arquivos_E) & set(arquivos_F):
    caminho_E = os.path.join(pasta_E, arquivo3)
    caminho_F = os.path.join(pasta_F, arquivo3)
    
    df_E = pd.read_csv(caminho_E)
    df_F = pd.read_csv(caminho_F)
    df_concatenado = pd.concat([df_E, df_F], axis=0)
    
    caminho_saida3 = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\BPO_PEOPLE\{}".format(arquivo3)
    df_concatenado.to_csv(caminho_saida3, index=False)
    



##__ADSO BPO UR% Data__


__Link:__ https://datapower-va.bytelemon.com/bi/visit/7284236472998969349

__Dashboard:__ TP-LIS ADSEO Performance Data

__Left Tab:__ UR

__Table:__ Last table (UR Projet/Site Breakdown)

__Filtros:__ Dynamic dimension (Day time) // Dynamic dimensio (hour time) // Time ( __Same range of the dates same as the bot__)\


In [16]:
#This code part will read the UR file from the new source of ADSO:
#Look fo the file on the downlaod folder
UR_ADSO = [filename for filename in os.listdir(downloads_folder_path) if
                  filename.startswith("_UR Project_Site Breakdown -")]
 
# Lista para armazenar DataFrames
dataframes = []
 
for file in UR_ADSO:
    filepath = os.path.join(downloads_folder_path, file)
    df = pd.read_csv(filepath)  # Ajuste para read_excel(filepath) se forem arquivos .xlsx
    dataframes.append(df)
 
# Concatenar todos os DataFrames, eliminando colunas duplicadas
combined_df = pd.concat(dataframes, axis=0, ignore_index=True)
# Remover colunas duplicadas
combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]
# Salvar o DataFrame combinado em um novo arquivo
output_filepath = os.path.join(downloads_folder_path, "combined_file.csv")  # Ajuste para .xlsx se necessário
print('A guardar o merge do file de ADSO.')
combined_df.to_csv(fr"C:\\Users\\{username}\\{folder}\\BPO_ADSO_merged.csv", index=False)
 
 
UR_ADSO_merge = [filename for filename in os.listdir(downloads_folder_path) if
                  filename.startswith("BPO_ADSO_merged")]
#Loop to read the file and transform the data:
for jj in UR_ADSO_merge:
    file_path = os.path.join(downloads_folder_path, jj)
    df = pd.read_csv(file_path)
    df.rename(columns={'Time.1': 'dt'}, inplace=True)
    df['Formatted_Hour'] = pd.to_datetime(df['dt'], format='%H').dt.strftime('%H:%M:%S')
    df['Hour']= df['Time']  + ' ' + df['Formatted_Hour']
    df = df[['Hour','Moderator']]
    df['Talent_X']=0
    print( 'A guardar o file de BPO_ADSO.')
    df.to_csv(fr"C:\\Users\\{username}\\{folder}\\BPO_ADSO.csv", index=False)
#---------------------------------------------------------------------------------------------
 
# Path to the directory containing the multiple files
folder_path = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\BPO_PEOPLE"
folder_path1 = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\Validation\BPO_People" 
folder_path2 = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\Validation\BPO_People_ADSO"

# Path to the single file with data
data_file = fr"C:\\Users\\{username}\\{folder}\\BPO_ADSO.csv"
 
# Read the single file with data into a DataFrame
try:
    data_df = pd.read_csv(data_file)
    print(f"Successfully read 'BPO_ADSO': {data_file}")
except FileNotFoundError:
     
    print(f"Data file not found: {data_file}")
    raise
except pd.errors.EmptyDataError:
    print(f"Data file is empty: {data_file}")
    raise
except Exception as e:
    print(f"An error occurred while reading the data file: {e}")
    raise
 
# Initialize min_date and max_date
min_date = data_df['Hour'].min()
max_date = data_df['Hour'].max()
 
# Generate all dates between min_date and max_date
date_range = pd.date_range(start=min_date, end=max_date)
 
# Convert all dates in the range to the desired string format
date_strings = [date.strftime('%d%m%Y') for date in date_range]
 
print("---------------------------------------------------------------------------------------")
print(f"Date range in data file: {date_strings[0]} to {date_strings[-1]}")
 
# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):  # Check if the file is a CSV
        # Extract start date and end date from the filename
        try:
            start_date_str, end_date_str = filename.split(' ')[-1].split('.')[0].split('_')
        except ValueError:
            continue
 
        # Check if any date in the range matches the start or end date in the filename
         # Check if the start date is within the date range
        if start_date_str in date_strings:
            # Now check if the end date is within the range or just beyond the max date
            if end_date_str in date_strings or pd.to_datetime(end_date_str, format='%d%m%Y') > pd.to_datetime(max_date):
                print(f"File {filename} matches the date range.")
            file_path = os.path.join(folder_path, filename)
            file_path2 = os.path.join(folder_path1, filename)
            try:
                # Read the current file into a DataFrame
                file_df = pd.read_csv(file_path)
               
            except FileNotFoundError:
                print(f"File not found: {file_path}")
                continue
            except pd.errors.EmptyDataError:
                print(f"File is empty: {file_path}")
                continue
            except Exception as e:
                print(f"An error occurred while reading the file: {e}")
                continue
           
   # Drop the Talent_X column if it exists
            if 'Talent_X' in file_df.columns:
                file_df.drop(columns=['Talent_X'], inplace=True)
 
            # Process the file
            merged_df = file_df.merge(data_df, on=['Hour', 'Moderator'], how='left')
            merged_df['Talent_X'] = merged_df['Talent_X'].fillna(1).astype(int)
           # Identify the index of rows where column B is 0
            index_to_drop = merged_df[merged_df['Talent_X'] == 0].index
            merged_df = merged_df.drop(index_to_drop)
            print('Foi feita o drop de linhas com 0 na columna "TalentX". ')      
                       
            # Salvar o DataFrame resultante de volta no arquivo original
            merged_df.to_csv(file_path, index=False)
            #merged_df.to_csv(file_path1, index=False)
            print("---------------------------------------------------------------------------------------")
            print(f"Successfully processed and saved file: {file_path}")
            print("---------------------------------------------------------------------------------------")
            print("----------------------NEXT FILE OR END-----------------------------------------------------------------")
 
#---------------------------------------------------------------------------------------------
UR_ADSO_Main = [filename for filename in os.listdir(downloads_folder_path) if
                  filename.startswith("_UR Project_Site Breakdown -")]
for jj in UR_ADSO_Main:
    file_path = os.path.join(downloads_folder_path, jj)
    df = pd.read_csv(file_path)
    df.rename(columns={'Time.1': 'dt'}, inplace=True)
    df['Formatted_Hour'] = pd.to_datetime(df['dt'], format='%H').dt.strftime('%H:%M:%S')
    df['Hour']= df['Time']  + ' ' + df['Formatted_Hour']
    df['Offline Hours(h)'] =0
    df['Others Hours(h)'] =0
    df = df.rename(columns ={"Utilization Rate of Effective Prod Hours":"Utilization Rate of Effective Prod Hours(%)","Occupancy rate":"Occupancy rate(%)","Shrinkage rate":"Shrinkage rate(%)",
    "Avg TCS EPH":"Effective Prod Hours(h)","Avg Mod. Task Hour":"Mod. Task Hours(h)","Avg Non-Mod Task Hour":"Non-mod. Task Hours(h)","Avg Meeting Hour":"Meeting Hours(h)","Avg Training Hour":"Training Hours(h)",
    "Avg Wellness Hours":"Wellness Hour(h)","Avg Rest Hour":"Rest Hours(h)","Avg Idle Hour":"Idle Hours(h)"})
    agents = df[['Hour','Moderator','Department Name','Effective Prod Hours(h)','Utilization Rate of Effective Prod Hours(%)','Shrinkage rate(%)','Occupancy rate(%)','Mod. Task Hours(h)','Non-mod. Task Hours(h)',
    'Wellness Hour(h)','Training Hours(h)','Meeting Hours(h)','Rest Hours(h)','Idle Hours(h)','Offline Hours(h)','Others Hours(h)']]
    agents['Talent_X']=0
    # Get Start Date for Each File
    start_date = pd.to_datetime(agents['Hour'].min())
    # Store the Year + Month of Start Date for End Of Month Calculation
    # And store the end date for 4 days and 8 days type file
    start_date_year = start_date.year
    start_date_month = start_date.month
    end_date1 = start_date + timedelta(days = 2)
    start_date = start_date.strftime('%d%m%Y')
    end_date1 = end_date1.strftime('%d%m%Y')
    # Normalizes the Start and End Date formats
    # Normalization of End Date Month on file specific Step
    if len(start_date) < 8:
        start_date = '0'+ start_date
    if len(end_date1) < 8:
        end_date1 = '0'+ end_date1
#-----------------------------------END of Dates logic----------------------------------------
    new_name = "WorkHour Indicators ADSO %s_%s.csv" %(start_date, end_date1)
    print("A guardar os files originais na pasta EMEA.")
    agents.to_csv(folder_path + '\\'+ new_name, index = False)
    agents.to_csv(folder_path2 + '\\'+ new_name, index = False)

A guardar o merge do file de ADSO.
A guardar o file de BPO_ADSO.
Successfully read 'BPO_ADSO': C:\\Users\\sequeira.81\\Downloads\\BPO_ADSO.csv
---------------------------------------------------------------------------------------
Date range in data file: 25082024 to 05092024
File WorkHour Indicators 03092024_05092024.csv matches the date range.
Foi feita o drop de linhas com 0 na columna "TalentX". 
---------------------------------------------------------------------------------------
Successfully processed and saved file: \\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\BPO_PEOPLE\WorkHour Indicators 03092024_05092024.csv
---------------------------------------------------------------------------------------
----------------------NEXT FILE OR END-----------------------------------------------------------------
File WorkHour Indicators 25082024_27082024.csv matches the date range.
Foi

# The below code will move the last modifiy files from BPO People to a databricks, and sharepoint, folder:

In [17]:
import os
import shutil
from datetime import datetime

def move_files_modified_today(BPO_folder, databricks_folder, BPO_Onedrive):
    # Get today's date
    today = datetime.today().date()

    # Get list of all files in the source directory
    files = [f for f in os.listdir(BPO_folder) if os.path.isfile(os.path.join(BPO_folder, f))]

    # Filter files modified today
    files_modified_today = []
    for file in files:
        file_path = os.path.join(BPO_folder, file)
        file_mod_time = datetime.fromtimestamp(os.path.getmtime(file_path)).date()

        if file_mod_time == today:
            files_modified_today.append(file)

    if not files_modified_today:
        print("No files modified today found in the source folder.")
        return
    
    # Move the files modified today
    for file in files_modified_today:
        src_file = os.path.join(BPO_folder, file)
        dst_file_databricks = os.path.join(databricks_folder, file)
        dst_file_onedrive = os.path.join(BPO_Onedrive, file)
        
        # Move to databricks_folder
        shutil.copy(src_file, dst_file_databricks)
        print(f"1-Copied'{file}' to '{databricks_folder}'.")

        # Copy to BPO_Onedrive
        shutil.copy(src_file, dst_file_onedrive)
        print(f"2-Copied '{file}' to '{BPO_Onedrive}'.")
    
# Example usage
BPO_folder = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\37. New DP Data\BPO_PEOPLE"
databricks_folder = r"\\10.235.133.90\ptdoufs01Departments$\ITDEV\DataManagement\TTOK\CONT\bpopeopleworkhour"
BPO_Onedrive = fr"C:\Users\{os.getenv('USERNAME')}\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\BPO_PEOPLE"

move_files_modified_today(BPO_folder, databricks_folder, BPO_Onedrive)


1-Copied'WorkHour Indicators 03092024_05092024.csv' to '\\10.235.133.90\ptdoufs01Departments$\ITDEV\DataManagement\TTOK\CONT\bpopeopleworkhour'.
2-Copied 'WorkHour Indicators 03092024_05092024.csv' to 'C:\Users\sequeira.81\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\BPO_PEOPLE'.
1-Copied'WorkHour Indicators 25082024_27082024.csv' to '\\10.235.133.90\ptdoufs01Departments$\ITDEV\DataManagement\TTOK\CONT\bpopeopleworkhour'.
2-Copied 'WorkHour Indicators 25082024_27082024.csv' to 'C:\Users\sequeira.81\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\BPO_PEOPLE'.
1-Copied'WorkHour Indicators 28082024_30082024.csv' to '\\10.235.133.90\ptdoufs01Departments$\ITDEV\DataManagement\TTOK\CONT\bpopeopleworkhour'.
2-Copied 'WorkHour Indicators 28082024_30082024.csv' to 'C:\Users\sequeira.81\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\BPO_PEOPLE'.
1-Copied'WorkHour Indicators 31

----------------------------------------END of this code Group--------------------------------------------------------------


__The below code will verify the folders of the UTC files and valiudate if exist some duplication of data:__

In [18]:
#7 Group
#THIS CODE WILL VERIFY EACH FOLDER AND EACH FILE AND GRAB AL THE DATES AND DO A UNIQUE DATE,
#IN CASE 2 FILE WILL HAVE THE SAME DATE THAT FILE AND DATE WILL BE SHOW ON THE CODE RESULT.
import os
import pandas as pd

def check_duplicate_dates_in_folder(folder_path):
    # List all files in the folder and sort them by modification time
    files = sorted(os.listdir(folder_path), key=lambda x: os.path.getmtime(os.path.join(folder_path, x)), reverse=True)[:5]
    # Create a dictionary to store unique dates from each file
    unique_dates_dict = {}
    # Iterate over the files
    for file in files:
        # Check if the file is a regular file (not a directory)
        if os.path.isfile(os.path.join(folder_path, file)):
            # Read the file content into a pandas DataFrame
            file_path = os.path.join(folder_path, file)
            try:
                df = pd.read_csv(file_path)  # Assuming files are in CSV format, adjust the format as needed
                # Check if the 'Date' or 'Hour' column exists in the DataFrame
                date_column = None
                if 'Date' in df.columns:
                    date_column = df['Date']
                elif 'Hour' in df.columns:
                    date_column = df['Hour']
                else:
                    raise ValueError(f"No 'Date' or 'Hour' found in the file: {file_path}")
                # Convert the date column to datetime
                df['Date'] = pd.to_datetime(date_column, errors='coerce')
                # Remove rows with invalid dates
                df = df.dropna(subset=['Date'])
                # Add unique dates to the dictionary
                unique_dates_dict[file] = set(df['Date'].unique())
            except pd.errors.EmptyDataError:
                pass  # Ignore empty files
            
    # Compare unique dates across all files to find duplicate dates
    duplicate_dates = set()
    for file1, dates1 in unique_dates_dict.items():
        for file2, dates2 in unique_dates_dict.items():
            if file1 != file2 and dates1 & dates2:
                duplicate_dates.update(dates1 & dates2)

    return duplicate_dates

for short_name, folder_path in folders2.items():
    duplicate_dates = check_duplicate_dates_in_folder(folder_path)
    if duplicate_dates:
        print(f'In folder {short_name}, the following dates are duplicated in at least two files: {duplicate_dates}.')
    else:
        print(f'In folder {short_name}, no duplicate dates were found among the files.')


In folder BPO_PEOPLE(3 em 3)download last 2 weeks data), no duplicate dates were found among the files.
In folder BPO_PEOPLE_ADSO(3 em 3)download last 2 weeks data), no duplicate dates were found among the files.
In folder BPO_QUEUE_HOUR_RR1.0 (4 em 4), no duplicate dates were found among the files.
In folder BPO_QUEUE_HOUR_RR2.0(4 em 4), no duplicate dates were found among the files.
In folder BPO_QUEUE_VMS(8 em 8), no duplicate dates were found among the files.
In folder MODERATION_STATS_HOUR_EMEA(4 em 4), no duplicate dates were found among the files.
In folder BPO_QUEUE_DAILY_RR1.0(8 em 8), no duplicate dates were found among the files.
In folder BPO_QUEUE_DAILY_RR2.0(8 em 8), no duplicate dates were found among the files.
In folder MODERATION_STATS_EMEA(15 em 15), no duplicate dates were found among the files.
In folder INTEGRITY_EMEA(15 em 15), no duplicate dates were found among the files.
In folder INTEGRITY_MI(15 em 15), no duplicate dates were found among the files.


----------------------------------------END of this code Group--------------------------------------------------------------


In [19]:
date=datetime.today().strftime('%d-%m-%Y %H-%M-%S')
Log_file=log_df.to_excel(fr"C:\\Users\\{username}\\{folder}\\Logs_etl_part1_{date}.xlsx", index=False)

--------------------------------------------------------------------------------------------------------------------------

# Shift data MMP


Link: https://byteworks-va.bytelemon.com/v2/workhour/correct

__Filters:__ By shift period

__Date range of the shift start date:__ 1 file per month, until day 15 of current month take the previous and the current month. 

__Data aggregattion method:__ By department

__Department:__ All withou VLN

In [25]:
shift_emea = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\39. MMP Shift Hours"
shift_Onedrive= fr"C:\Users\\{username}\\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\39. MMP Shift Hours"
# Look for the file starting with "Project Omega RCA" in the source directory
source_dir =fr"C:\\Users\\{username}\\{folder}"
file_prefix = "data-"

file_to_move = next((file for file in os.listdir(source_dir) if file.startswith(file_prefix)), None)
file_path = os.path.join(source_dir, file_to_move)

u = pd.read_csv(file_path,skiprows = 1)
    #Change the date to string:
  
start_date = pd.to_datetime(u['日期/Date'].min())
start_date_year = start_date.year
start_date_month = start_date.month

final_day = calendar.monthrange(start_date_year, start_date_month)[1]
len_start_date_month = str(start_date_month)
if len(len_start_date_month) < 2:
    len_start_date_month = '0' + len_start_date_month    

start_date_str = start_date.strftime('%d%m%Y')
end_date_str = "%s%s%s" % (final_day, len_start_date_month, start_date_year)

if len(start_date_str) < 8:
    start_date_str = '0' + start_date_str 


if file_to_move:
    try:
      
        # Define the new file name
        new_file_name = 'Shift_data_MMP-%s_%s.csv' % (start_date_str, end_date_str)  # Replace with the desired new file name and extension
        
        # Move and rename the file
        shutil.copy(os.path.join(source_dir, file_to_move), os.path.join(shift_emea, new_file_name))
        print(f"File '{file_to_move}' moved and renamed to '{new_file_name}' in '{shift_emea}'")

        # Move  and rename the file in sharepoint
        shutil.copy(os.path.join(source_dir, file_to_move), os.path.join(shift_Onedrive, new_file_name))
        print(f"File '{file_to_move}' moved and renamed to '{new_file_name}' in '{shift_Onedrive}'")

    except Exception as e:
        print(f"Error occurred while moving the file: {e}")
else:
    print(f"No file starting with '{file_prefix}' found in '{source_dir}'")

#This line will remove the files from your download files:
[os.remove(os.path.join(downloads_folder_path, filename)) for filename in os.listdir(downloads_folder_path)
 if filename.startswith("data-")]

File 'data-1725607099_1725607167284.csv' moved and renamed to 'Shift_data_MMP-01092024_30092024.csv' in '\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\39. MMP Shift Hours'
File 'data-1725607099_1725607167284.csv' moved and renamed to 'Shift_data_MMP-01092024_30092024.csv' in 'C:\Users\\sequeira.81\\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\39. MMP Shift Hours'


[None, None]

# OMEGA Client


Link: https://datapower-va.bytelemon.com/bi/visit/7325384900452941829?immersive=1

Downloads :On Monday download previous week,after that download current week. ( EVERYDAY )

__Tab:__ Project Omega

__Batch Date:__ Select the week from Monday to Sunday

__Moderator Base:__ TP-LIS

Click on 3 dots from the table, select download

UTF-8 encoded CSV, another 0 on "Number of rows" and Download

In [20]:
shift_emea = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\9. Quality Data\SV OMEGA Project\Client_Data"
# Look for the file starting with "Project Omega RCA" in the source directory
source_dir =fr"C:\\Users\\{username}\\{folder}"
file_prefix = "_Detailed Mod Level Occurence Table   -"

file_to_move = next((file for file in os.listdir(source_dir) if file.startswith(file_prefix)), None)
file_path = os.path.join(source_dir, file_to_move)

u = pd.read_csv(file_path)
# Define the source and destination directories

start_date = pd.to_datetime(u['Resolve Time'].min())
end_date = start_date + timedelta(days = 6)

start_date = start_date.strftime('%d%m%Y')
end_date = end_date.strftime('%d%m%Y')   



if file_to_move:
    try:
      
        # Define the new file name
        new_file_name = '_Detailed Mod Level Occurence Table   -%s_%s.csv' % (start_date, end_date)  # Replace with the desired new file name and extension
        
        # Move and rename the file
        shutil.copy(os.path.join(source_dir, file_to_move), os.path.join(shift_emea, new_file_name))
        print(f"File '{file_to_move}' moved and renamed to '{new_file_name}' in '{shift_emea}'")
    except Exception as e:
        print(f"Error occurred while moving the file: {e}")
else:
    print(f"No file starting with '{file_prefix}' found in '{source_dir}'")

File '_Detailed Mod Level Occurence Table   - 2024-09-06 08-19-36.csv' moved and renamed to '_Detailed Mod Level Occurence Table   -26082024_01092024.csv' in '\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\9. Quality Data\SV OMEGA Project\Client_Data'


# OMEGA RCAs



Link: https://teleperformance.larksuite.com/sheets/Hkhis4qirhViP6t3dZguQPZzsyc?sheet=tYlF1U

Downloads : Monday, previous week,after current week. ( EVERYDAY )

__STEPS:__ 3 Dots, Download As, CSV

In [21]:
shift_emea = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\9. Quality Data\SV OMEGA Project\Support_File"
# Look for the file starting with "Project Omega RCA" in the source directory
source_dir =fr"C:\\Users\\{username}\\{folder}"
file_prefix = "Project Omega RCA - 2024 - "

file_to_move = next((file for file in os.listdir(source_dir) if file.startswith(file_prefix)), None)
file_path = os.path.join(source_dir, file_to_move)

u = pd.read_csv(file_path)

if file_to_move:
    try:
      
     
        
        # Move and rename the file
        shutil.copy(os.path.join(source_dir, file_to_move), os.path.join(shift_emea))
        print(f"File '{file_to_move}' moved  in '{shift_emea}'")
    except Exception as e:
        print(f"Error occurred while moving the file: {e}")
else:
    print(f"No file starting with '{file_prefix}' found in '{source_dir}'")

File 'Project Omega RCA - 2024 - W35.csv' moved  in '\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\9. Quality Data\SV OMEGA Project\Support_File'


# OMEGA Script


In [22]:
Script_path = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\10. Python scripts\NEW ETL\Aux Scripts\Omega Project\omega_script.py"

Result1 = subprocess.run(['python', Script_path], text=True)

print("Result 1:", Result1)


Result 1: CompletedProcess(args=['python', '\\\\emea.tpg.ads\\portugal\\Departments\\ITDEV\\PowerBI\\accounting\\business analysts\\01 - ba\\02 - projects\\tiktok\\10. Python scripts\\NEW ETL\\Aux Scripts\\Omega Project\\omega_script.py'], returncode=0)


--------------------------------------------------------------------------------------------------------------------------

# MI RockAppeal




__Link:__ https://rock-va.bytelemon.com/appeal_center/list/claimantStage/unhandled?case_merge=&verifier_type=0&case_state=0&case_content=&search_value_enum=0&origin_audit_time=&new_audit_time=&order_field=remain_time&order=0&origin_sample_index=%5b%5d&new_sample_index=%5b%5d&origin_verifier=&appeal_mode=&task_id=&object_id=&start=0&end=100&role=4

__TAB:__ Claimant

__TAB:__ __Pending Appeal__ No need to select dates as the file is small and can contain all the data. 

__TAB:__ __Appeal__  On the filter Moderation time we need to extract (__curent month and previous month__), then click on the left box with a up arrow // Online Export //
Whether separate sheets based on the queues ( __select: NO__) // Apply // name as __2 and 3__ a number for each month downloaded.

__TAB:__ __Not Appeal__  On the filter Moderation time we need to extract (__curent month and previous month__), then click on the left box with a up arrow // Online Export //
Whether separate sheets based on the queues ( __select: NO__) // Apply // name as __4 and 5__ a number for each month downloaded.

__TAB:__ __Expired__  On the filter Moderation time we need to extract (__curent month and previous month__), then click on the left box with a up arrow // Online Export //
Whether separate sheets based on the queues ( __select: NO__) // Apply // name as __6 and 7__ a number for each month downloaded.

After all that we need to go to the left box with a up arrow // My Exports// For each file will pop-up a new lark window and then you need to download the 7 files in .CSV

All the files downloaded, then we can run the bellow code.

In [23]:

#Destination Folder:
MI_Pending_Appealed= r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\10. MI\Labeling_Rock_Appeal\Pending Appeals"
MI_Not_Appeal= r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\10. MI\Labeling_Rock_Appeal\Not Appeal"
MI_Expired= r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\10. MI\Labeling_Rock_Appeal\Expired"
MI_Appealed= r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\10. MI\Labeling_Rock_Appeal\Appealed"
MI_Pending_Appealed_OneDrive= fr"C:\Users\{os.getenv('USERNAME')}\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\Labeling_Rock_Appeal\Pending Appeals"
MI_Not_Appeal_OneDrive= fr"C:\Users\{os.getenv('USERNAME')}\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\Labeling_Rock_Appeal\Not Appeal"
MI_Expired_OneDrive= fr"C:\Users\{os.getenv('USERNAME')}\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\Labeling_Rock_Appeal\Expired"
MI_Appealed_OneDrive= fr"C:\Users\{os.getenv('USERNAME')}\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\Labeling_Rock_Appeal\Appealed"


# Caminho para a pasta de downloads
folder_path = fr"C:\\Users\\{username}\\{folder}"

# Função para renomear os arquivos

    # Lista todos os arquivos na pasta de downloads
files = os.listdir(folder_path)

for filename in files:
        # Verifica se o arquivo é um Excel
        if filename.endswith('- sheet.csv'):
            # Lê a data mínima da coluna '日期/Date'
            df = pd.read_csv(os.path.join(folder_path, filename))
            start_date = pd.to_datetime(df['Moderation Time'].min())
            start_date_year = start_date.year
            start_date_month = start_date.month

            # Calcula o último dia do mês
            final_day = calendar.monthrange(start_date_year, start_date_month)[1]
            len_start_date_month = str(start_date_month).zfill(2)

            # Formata as datas de início e fim
            start_date_str = start_date.strftime('%Y%m%d')
            end_date_str = f"{start_date_year}{len_start_date_month}{final_day}"

            # Adiciona um zero se necessário
            if len(start_date_str) < 8:
                start_date_str = '0' + start_date_str

            # Renomeia o arquivo com base na lógica fornecida
           # Rename the file based on the provided logic
        if filename == '1.xlsx - sheet.csv':
            f = 'Rock_Appeal_Labeling_PendingAppeals.csv'
            df.to_csv(os.path.join(MI_Pending_Appealed, f), index=False)
            df.to_csv(os.path.join(MI_Pending_Appealed_OneDrive, f), index=False)
            print(f"File saved as {f}")

        elif filename in ['2.xlsx - sheet.csv', '3.xlsx - sheet.csv']:
            f = f"Rock_Appeal_Labeling_Appealed_{start_date_str}_{end_date_str}.csv"
            df.to_csv(os.path.join(MI_Appealed, f), index=False)
            df.to_csv(os.path.join(MI_Appealed_OneDrive, f), index=False)
            print(f"File saved as {f}")

        elif filename in ['4.xlsx - sheet.csv', '5.xlsx - sheet.csv']:
            f = f"Rock_Appeal_Labeling_NotAppeal_{start_date_str}_{end_date_str}.csv"
            df.to_csv(os.path.join(MI_Not_Appeal, f), index=False)
            df.to_csv(os.path.join(MI_Not_Appeal_OneDrive, f), index=False)
            print(f"File saved as {f}")

        elif filename in ['6.xlsx - sheet.csv', '7.xlsx - sheet.csv']:
            f = f"Rock_Appeal_Labeling_Expired_{start_date_str}_{end_date_str}.csv"
            df.to_csv(os.path.join(MI_Expired, f), index=False)
            df.to_csv(os.path.join(MI_Expired_OneDrive, f), index=False)
            print(f"File saved as {f}")
                
    


File saved as Rock_Appeal_Labeling_PendingAppeals.csv
File saved as Rock_Appeal_Labeling_Appealed_20240901_20240930.csv
File saved as Rock_Appeal_Labeling_Appealed_20240801_20240831.csv
File saved as Rock_Appeal_Labeling_NotAppeal_20240901_20240930.csv
File saved as Rock_Appeal_Labeling_NotAppeal_20240801_20240831.csv
File saved as Rock_Appeal_Labeling_Expired_20240901_20240930.csv
File saved as Rock_Appeal_Labeling_Expired_20240801_20240831.csv


# <del>MI Labeling LP Platform</del>

__Link:__ https://label.bytedance.com/operation/data-dashboards?device_id=81889&org=DCC
        
__TAB:__ Queue List

__TAB:__ Data Dashboard

__Table:__ Labeller Project Mapping

__Filter:__ Resolve date ( The week you want from Monday to Sunday)

If Monday download previous week, starting Tuesday forward current week (EVERYDAY)

Click on 3 dots from the table, select download 

UTF-8 encoded CSV, anotehr 0 on "Number of rows" and Download

In [61]:
# Define the source folder (downloads) and target folder
source_folder = downloads_folder_path
target_folder = fr"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\10. MI\Labeling_LP_Platform\Weekly_data" 

# Determine the current day and calculate the date range
today = datetime.today()
weekday = today.weekday()

if weekday == 0:  # Monday
    # Use previous week's range
    start_date = today - timedelta(days=7)
else:
    # Use current week's range
    start_date = today - timedelta(days=weekday)

end_date = start_date + timedelta(days=6)

# Format the dates as YYYYMMDD
start_date_str = start_date.strftime('%Y%m%d')
end_date_str = end_date.strftime('%Y%m%d')

# Construct the new file name
new_file_base = f"_Labeller Project Mapping_{start_date_str}_{end_date_str}.csv"

# Find all files matching the pattern in the source folder
file_pattern = "_Labeller Project Mapping*.csv"  # Adjust extension if different
matching_files = glob.glob(os.path.join(source_folder, file_pattern))

if matching_files:
    for latest_file in matching_files:
        try:
            # Rename the file with the new date range
            new_file_name = new_file_base
            new_file_path = os.path.join(source_folder, new_file_name)

            # Rename the file
            os.rename(latest_file, new_file_path)
            print(f"Renamed: {os.path.basename(latest_file)} to {new_file_name}")

            # Check if a file with the same date range exists in the target folder
            existing_file = None
            for file in os.listdir(target_folder):
                if new_file_base in file:
                    existing_file = file
                    break

            if existing_file:
                existing_file_path = os.path.join(target_folder, existing_file)
                os.remove(existing_file_path)
                log_df = log_df.append({'Filename': existing_file, 'Date': date_today, 'Status': 'Replaced', 'Message': 'Existing file replaced'}, ignore_index=True)
                print(f"Replaced: {existing_file}")
            
            # Move the renamed file to the target folder
            shutil.copy(new_file_path, os.path.join(target_folder, new_file_name))
            log_df = log_df.append({'Filename': new_file_name, 'Date': date_today, 'Status': 'Moved', 'Message': 'File moved successfully'}, ignore_index=True)
            print(f"Moved: {new_file_name} to {target_folder}")

        except Exception as e:
            log_df = log_df.append({'Filename': os.path.basename(latest_file), 'Date': date_today, 'Status': 'Error', 'Message': str(e)}, ignore_index=True)
            print(f"Error: {e}")
else:
    print("No files found matching the pattern.")


Renamed: _Labeller Project Mapping - 2024-08-23 14-19-17.csv to _Labeller Project Mapping_20240819_20240825.csv
Replaced: _Labeller Project Mapping_20240819_20240825.csv
Moved: _Labeller Project Mapping_20240819_20240825.csv to C:\Users\Sequeira.81\Desktop\New folder


# MI Labeling LP Platform Daily

__Link:__ https://teleperformance.sharepoint.com/sites/TikTok212/Shared%20Documents/Forms/AllItems.aspx?FolderCTID=0x012000B98B5AF80497F14392E44394E54AA19B&isAscending=true&id=%2Fsites%2FTikTok212%2FShared%20Documents%2FData%20Analytics%2F5%2E%20Extractions%2FLabeling%20LP%20Platform&sortField=LinkFilename&viewid=4d19ea0e%2D0774%2D4a0a%2D8fc4%2D204e3ee1184f
__INFO:__ THIS LINK IS IN CASE YOU NEED TO DO IT MANUALLY

In [28]:
# Clean Python cache
gen_py_path = fr"C:\Users\\{username}\\AppData\Local\Temp\gen_py"
shutil.rmtree(gen_py_path, ignore_errors=True)

# Define the source and destination folders
source_folder = r"C:\Users\Sequeira.81\Teleperformance\P.TTOK.CONT - 5. Extractions\Labeling LP Platform"
destination_folder = r"\\emea.tpg.ads\portugal\Departments\ITDEV\PowerBI\accounting\business analysts\01 - ba\02 - projects\tiktok\4. Raw Data and Aux Files\10. MI\Labeling_LP_Platform\Daily_data"
#destination_folder_2 = r"C:\Users\Sequeira.81\Teleperformance\pt-dataanalytics-pbi-datadrops - P.TTOK.CONT\4. Raw Data and Aux Files\Labeling_LP_Platform\Daily_data"

# Log DataFrame
log_df = pd.DataFrame(columns=['Filename', 'Date', 'Status', 'Message'])

# Get the list of files in the source folder
files = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]

# Check for non-CSV files
non_csv_files = [f for f in files if not f.endswith('.csv')]
if non_csv_files:
    # Create and send an email notification for non-CSV files
    ol = win32com.client.Dispatch("outlook.application")
    olmailitem = 0x0
    newmail = ol.CreateItem(olmailitem)
    newmail.Subject = 'Non-CSV Files Found in Labeling LP Platform Folder'
    newmail.To = 'iuliia.kritskaia@pt.teleperformance.com'
    newmail.Body = f"Hello, the following non-CSV files were found in the Labeling LP Platform folder:\n\n" + "\n".join(non_csv_files)
    newmail.Send()

# Filter files that start with "_Labeller Project Mapping" and end with ".csv"
filtered_files = [f for f in files if f.startswith("_Labeller Project Mapping") and f.endswith(".csv")]

# Sort files by modification time (most recent first)
filtered_files.sort(key=lambda f: os.path.getmtime(os.path.join(source_folder, f)), reverse=True)

# Get the last 5 files
latest_files = filtered_files[:5]

# Move the last 5 files to the destination folder
for file_name in latest_files:
    try:
        # Build full file path
        source_path = os.path.join(source_folder, file_name)
        destination_path = os.path.join(destination_folder, file_name)
        #destination_path_2 = os.path.join(destination_folder_2, file_name)
        
        # Move the file
        shutil.copy(source_path, destination_path)
        #shutil.copy(source_path, destination_path_2)
        
        # Log success
        log_df = log_df.append({
            'Filename': file_name,
            'Date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'Status': 'Success',
            'Message': 'File moved successfully'
        }, ignore_index=True)
    except Exception as e:
        # Log failure
        log_df = log_df.append({
            'Filename': file_name,
            'Date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'Status': 'Failure',
            'Message': str(e)
        }, ignore_index=True)

# Print log DataFrame
print("---------------------------------------------")
print(log_df)


---------------------------------------------
                                 Filename                 Date   Status  \
0  _Labeller Project Mapping_20240904.csv  2024-09-06 12:37:41  Success   
1  _Labeller Project Mapping_20240905.csv  2024-09-06 12:37:41  Success   
2  _Labeller Project Mapping_20240903.csv  2024-09-06 12:37:41  Success   
3  _Labeller Project Mapping_20240902.csv  2024-09-06 12:37:42  Success   
4  _Labeller Project Mapping_20240831.csv  2024-09-06 12:37:42  Success   

                   Message  
0  File moved successfully  
1  File moved successfully  
2  File moved successfully  
3  File moved successfully  
4  File moved successfully  
